In [1]:
# importing function from a different ipny 
import pandas as pd
import collections
from keras.initializers import Constant
import numpy as np
import matplotlib.pyplot as plt
import re
import os
import operator
from itertools import product
import sys
import numpy as np
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from functools import reduce
import spacy
from spacy.tokenizer import Tokenizer
nlp = spacy.load("en_core_web_sm")
tokenizer = Tokenizer(nlp.vocab)
pd.options.display.max_columns = 1000
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Activation, Input, BatchNormalization, MaxPooling1D, Bidirectional,LSTM
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPool1D, Flatten , Embedding, GlobalMaxPool1D
from keras.models import Model
from keras.wrappers.scikit_learn import KerasClassifier
#https://www.amazon.com/Neural-Networks-Deep-Learning-Textbook/dp/3319944622/ref=cm_cr_arp_d_product_top?ie=UTF8
#https://towardsdatascience.com/activation-functions-neural-networks-1cbd9f8d91d6

/home/titli/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

MAX_NUM_WORDS = 20000

In [3]:
keras_data = pd.read_csv('subject_spam.csv', index_col=0, encoding='utf8', engine='python')
keras_data.fillna("_na_", inplace = True)
keras_data = keras_data.drop(keras_data[keras_data.isspam == "_na_"].index)
spammer = {'spam ': 1,'ham ': 0} 
keras_data.isspam = [spammer[item] for item in keras_data.isspam] 
spam_index = keras_data[keras_data.isspam ==1].index
ham_index = keras_data[keras_data.isspam == 0].index
new_index = np.concatenate((spam_index[:10000], ham_index[:9997]), axis=0)
keras_data_new = keras_data.iloc[new_index]
labels = keras_data_new.isspam
keras_data_new.head()

,Subject,isspam
0,"Confidential :Soma:, Ci@lis, :P:ntermin",1
1,¢Â ÇªÁüÇÑ ¼³³¯ ÀÌº¥Æ® ÀÀ¸ðÇØ¼­ ºÎ¸ð´Ô²² ¼±¹°ÇÏ...,1
2,_na_,1
3,StOck 0ppurtunities - their sh0Oting,1
6,All your prescr[iption needs right here,1


In [4]:
VALIDATION_SPLIT = 0.2
MAX_SEQUENCE_LENGTH = 1000
EMBEDDING_DIM = 100

In [5]:
embeddings_index = {}
with open(os.path.join('glove.6B.100d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

# second, prepare text samples and their labels
print('Processing text dataset')

Found 400000 word vectors.
Processing text dataset


In [6]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(keras_data_new.Subject)
sequences = tokenizer.texts_to_sequences(keras_data_new.Subject)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]



Found 15425 unique tokens.
Shape of data tensor: (19997, 1000)
Shape of label tensor: (19997, 2)


In [7]:
print('Preparing embedding matrix.')

# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

print('Training model.')



Preparing embedding matrix.
Training model.


In [8]:
def loss_edit(out):
    loss_list = [s for s in out.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in out.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in out.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in out.history.keys() if 'acc' in s and 'val' in s]
    train_loss = []
    val_loss = []
    train_acc = []
    val_acc = []
    for l in loss_list:
        train_loss .append(out.history[l])
    for l in val_loss_list:
        val_loss .append(out.history[l])
    for l in acc_list:
        train_acc.append(out.history[l])
    for l in val_acc_list:
        val_acc .append(out.history[l])
    line  = { 'train_loss': reduce(operator.concat, train_loss)[-1],'val_loss': reduce(operator.concat, val_loss)[-1], \
             'train_acc': reduce(operator.concat, train_acc)[-1], 'val_acc': reduce(operator.concat, val_acc)[-1] }
    return (line)

# LSTM Model

In [9]:
def embedding_LSTM(fl1=16, fl2=16, fl3=16, dl=16, optimizer= 'RMSprop', kl = 5, layer =1): 
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    x = Bidirectional(LSTM(units = fl1, return_sequences=True))(embedded_sequences)
    x = GlobalMaxPool1D()(x)
    x = Dense(units=dl, activation="relu")(x)
    x = Dropout(0.1)(x)
    preds = Dense(2, activation='softmax')(x)

    model = Model(sequence_input, preds)
    model.compile(loss= 'categorical_crossentropy',
              optimizer= optimizer,
              metrics=['acc'])
    return model

In [11]:
layers = [1]
fl1s = [16,32,64, 128]
fl2s = [0]
fl3s = [0]
kls = [0]
dls = [16,32,64,128]
#sgd = optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
optimizers = ['RMSprop', 'Adam', 'SGD', 'Nadam'] #, 'Adadelta', 'Adam', 'Adamax', 'Nadam']
loss_data = []
params = []
for fl1, fl2, fl3, kl, dl, optimizer,layer in product(fl1s,fl2s,fl3s,kls,dls,optimizers,layers):
    kwargs = dict(fl1=fl1, fl2= fl2, fl3=fl3, kl=kl, dl=dl, optimizer= ''.join(optimizer), layer=layers)
    params.append(kwargs)
    model = embedding_LSTM(**kwargs)
    #model.fit(x_train, y_train)
    history = model.fit(x_train, y_train, batch_size= 16, epochs=2, validation_data=(x_val, y_val))
    loss_data.append(loss_edit(history))
    print(loss_edit(history))

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 1361s 85ms/step - loss: 0.3575 - acc: 0.8491 - val_loss: 0.2686 - val_acc: 0.8935
Epoch 2/2
15998/15998 [==============================] - 1282s 80ms/step - loss: 0.2497 - acc: 0.8979 - val_loss: 0.2354 - val_acc: 0.9035
{'train_loss': 0.24968781153044933, 'val_loss': 0.2354417315633722, 'train_acc': 0.8978622327790974, 'val_acc': 0.9034758689821467}
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 1254s 78ms/step - loss: 0.3523 - acc: 0.8503 - val_loss: 0.2630 - val_acc: 0.8970
Epoch 2/2
15998/15998 [==============================] - 1266s 79ms/step - loss: 0.2355 - acc: 0.9021 - val_loss: 0.2264 - val_acc: 0.9127
{'train_loss': 0.2354699908226546, 'val_loss': 0.22638070953729958, 'train_acc': 0.9021127641029635, 'val_acc': 0.9127281820455114}
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==========

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 1379s 86ms/step - loss: 0.6103 - acc: 0.7022 - val_loss: 0.4867 - val_acc: 0.8122
Epoch 2/2
15998/15998 [==============================] - 1305s 82ms/step - loss: 0.4308 - acc: 0.8191 - val_loss: 0.3787 - val_acc: 0.8427
{'train_loss': 0.43078653529474537, 'val_loss': 0.3786606387954618, 'train_acc': 0.8191023878059264, 'val_acc': 0.8427106776694173}
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 1340s 84ms/step - loss: 0.3047 - acc: 0.8702 - val_loss: 0.2365 - val_acc: 0.9017
Epoch 2/2
15998/15998 [==============================] - 1318s 82ms/step - loss: 0.1908 - acc: 0.9209 - val_loss: 0.1944 - val_acc: 0.9260
{'train_loss': 0.19078558231554324, 'val_loss': 0.19438855479690068, 'train_acc': 0.920927615951994, 'val_acc': 0.9259814953738434}
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==========

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - ETA: 2s - loss: 0.3236 - acc: 0.865 - 2890s 181ms/step - loss: 0.3237 - acc: 0.8650 - val_loss: 0.2960 - val_acc: 0.8782
Epoch 2/2
15998/15998 [==============================] - 1257s 79ms/step - loss: 0.2258 - acc: 0.9061 - val_loss: 0.2183 - val_acc: 0.9100
{'train_loss': 0.2257649445002087, 'val_loss': 0.21833431556966193, 'train_acc': 0.9061132641580197, 'val_acc': 0.9099774943884983}
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 1203s 75ms/step - loss: 0.3070 - acc: 0.8707 - val_loss: 0.2218 - val_acc: 0.9155
Epoch 2/2
10432/15998 [==================>...........] - ETA: 7:32 - loss: 0.1955 - acc: 0.9198


KeyboardInterrupt



In [23]:
params_pd = pd.DataFrame(params)
loss_pd= pd.DataFrame(loss_data)

In [25]:
embeddings_LSTM = pd.concat([params_pd, loss_pd], axis=1, join='inner')
embeddings_LSTM.to_csv('embeddings_LSTM_model.csv', index=False)

In [34]:
embeddings_LSTM_sort = embeddings_LSTM.sort_values(['val_loss'])
embeddings_LSTM_sort

,dl,fl1,fl2,fl3,kl,layer,optimizer,train_acc,train_loss,val_acc,val_loss
27,64,32,0,0,0,[1],Nadam,0.928179,0.180042,0.924231,0.182697
31,128,32,0,0,0,[1],Nadam,0.925866,0.178017,0.926732,0.188866
19,16,32,0,0,0,[1],Nadam,0.920928,0.190786,0.925981,0.194389
35,16,64,0,0,0,[1],Nadam,0.914677,0.205629,0.923231,0.196220
24,64,32,0,0,0,[1],RMSprop,0.911114,0.211151,0.919730,0.197624
7,32,16,0,0,0,[1],Nadam,0.920240,0.193405,0.920480,0.198082
11,64,16,0,0,0,[1],Nadam,0.922865,0.191926,0.923981,0.198908
15,128,16,0,0,0,[1],Nadam,0.919740,0.190584,0.923731,0.199320
29,128,32,0,0,0,[1],Adam,0.920428,0.194369,0.916729,0.200601
17,16,32,0,0,0,[1],Adam,0.912677,0.216945,0.923731,0.200625
